<a href="https://colab.research.google.com/github/Renad-Alruthea/DATAMINING-project/blob/main/Data_mining_%26_information_retriever_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os._exit(0)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import glob
from pathlib import Path

directory_path = '/content/drive/MyDrive/Colab Notebooks/Document'
TextFiles = glob.glob(f"{directory_path}/*.txt")
TextTitles = [Path(text).stem for text in TextFiles]

#Debug the file search
if not TextFiles:
    print("No text files found in the specified directory. Please verify the path and the files.")
else:
    print("Text files found:", TextFiles)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install --upgrade --force-reinstall numpy==1.26.0

In [ ]:
!pip install --upgrade --force-reinstall gensim


In [ ]:

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from gensim.utils import tokenize # This import should now work after reinstalling numpy and gensim
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
stopwords = nltk.corpus.stopwords.words('english')
import string

def normalize (text):
  t = tokenize(text.lower())
  s = [word for word in t if word not in stopwords]
  p = [word for word in s if word not in string.punctuation]
  st = [ps.stem(word) for word in p]
  l = [lemmatizer.lemmatize(word) for word in st]
  return l

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

In [ ]:

import os

original_docs = []
normalized_docs = []
from google.colab import drive
drive.mount("/content/drive")
import glob
from pathlib import Path

directory_path = '/content/drive/MyDrive/Colab Notebooks/Document'
TextFiles = glob.glob(f"{directory_path}/*.txt")
TextTitles = [Path(text).stem for text in TextFiles]
for filename in os.listdir(directory_path): # Use os.listdir to get files in the directory
    txt_file = os.path.join(directory_path, filename) # Construct the full file path
    if os.path.isfile(txt_file) and filename.endswith(".txt"): # Check if it's a file and ends with .txt
        with open(txt_file, encoding="utf-8") as f: # Open the file with utf-8 encoding
            txt_file_as_string = f.read()
            original_docs.append(txt_file_as_string)
            normalized_docs.append(normalize(txt_file_as_string))

print(*original_docs, sep='\n')
print(*normalized_docs, sep='\n')

In [ ]:

def counting_docs (w, docs):
  count = 0
  for r in range (len(docs)):
    if (w in docs[r]):
      count += 1
  return (count)

In [ ]:

import math
def Calc_TF_IDF (document, wordDict):
  TF_IDF = []
  for r in range(len(document)):
    row = []
    for w in wordDict:
      TF = document[r].count(w)/len(document[r])
      row.append (round(TF*Calc_IDF(w, document),4))
    TF_IDF.append(row)
  return (TF_IDF)

def Calc_IDF (w, documents):
  return math.log10((len(documents)/counting_docs(w, documents)))

In [ ]:

import pandas as pd
flat_list = []
for sublist in normalized_docs:
    for item in sublist:
        flat_list.append(item)
wordDict = list(dict.fromkeys(flat_list))

TF_IDF = Calc_TF_IDF(normalized_docs, wordDict)
# Assuming 'TextTitles' was intended instead of 'text_titles':
df = pd.DataFrame(TF_IDF, index=TextTitles, columns=wordDict)

df

In [ ]:

def Calc_Q_IDF(query, docs, wordDict):
    Q_IDF = []
    for w in wordDict:
      if (query.count(w) == 0):
        Q_IDF.append(float(0))
      else:
        Q_IDF.append(round(query.count(w) * Calc_IDF(w, docs), 3))
    return Q_IDF

In [ ]:
query = 'sugar'
normalized_query = normalize (query)
QTF = Calc_Q_IDF (normalized_query, normalized_docs, wordDict)

In [ ]:

import numpy as np

def cos_sim(a, b):
    mangitude_product = math.sqrt((np.dot(a, a) * np.dot(b, b)))
    if ( mangitude_product == 0):
      return 0
    else:
	    return np.dot(a, b) /  mangitude_product


In [ ]:

from operator import itemgetter

doc_sim = []

for r in range(len(normalized_docs)):
    cosine = cos_sim(QTF, TF_IDF[r])
    if (cosine != 0):
        doc_sim.append([TextTitles[r], cosine])

if (len(doc_sim) == 0):
    print("query not found")
else:
    sorted_list = sorted(doc_sim, key=itemgetter(1), reverse=True)
    print(*sorted_list, sep='\n')

In [ ]:
# 15 relevant , 15 irrelevant
df_labeled = df.reset_index()['index'].to_frame()
df_labeled['y'] = np.where((df_labeled['index'] == 'Sport')|(df_labeled['index'] == 'Art')|(df_labeled['index'] == 'Makeup')
|(df_labeled['index'] == 'Business')|(df_labeled['index'] == 'Saudi Arabia')|(df_labeled['index'] == 'Travel')|(df_labeled['index'] == 'Happiness')
|(df_labeled['index'] == 'ADHD')|(df_labeled['index'] == 'Cars')|(df_labeled['index'] == 'Environmental Pollution')|(df_labeled['index'] == 'Fashion')
|(df_labeled['index'] == 'furniture')|(df_labeled['index'] == 'Online Learning')|(df_labeled['index'] == 'Stealing')|(df_labeled['index'] == 'Technology') , 0,1)
df_labeled.rename(columns={"index": "Document"},inplace=True)
df_labeled

In [ ]:
df_pred = pd.DataFrame(sorted_list, columns=['Document', 'cos_sim'])
df_pred['y_pred'] = np.where(df_pred['cos_sim'] >= 0.05,1, 0)

In [ ]:
df_pred

In [ ]:
df_pred = df_pred.sort_values('Document',ascending=True)
df_labeled = df_labeled.sort_values('Document',ascending=True)

In [ ]:
pd.merge(df_labeled,df_pred,on='Document')

In [ ]:
df2 = pd.merge(df_labeled,df_pred,on='Document').drop('cos_sim',axis=1)
df2

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df2['y'], df2['y_pred'])
TN, FP, FN, TP = confusion_matrix(df2['y'], df2['y_pred']).ravel()
print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)

In [ ]:
Precision = TP / (TP + FP)
Recall = TP / (TP + FN)
print("Precision :" , Precision , "Recall: " , Recall)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

file_path = '/content/BakedFoodNutritions_With_Target.csv'
df = pd.read_csv(file_path)


df_copy = df.copy()


true_labels = df_copy['Target']

# حذف الأعمدة والهدف
drop_cols = ['Food', 'ProductType', 'FlavorVariant', 'MeasureType', 'Target']
X = df_copy.drop(columns=drop_cols)


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# تطبيق خوارزمية K means ,عدد الكلاستر 2
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
predicted_clusters = kmeans.fit_predict(X_scaled)


le = LabelEncoder()
true_labels_encoded = le.fit_transform(true_labels)

# حساب الدقة - نأخذ الأعلى بين التصنيفين
accuracy = max(
    accuracy_score(true_labels_encoded, predicted_clusters),
    accuracy_score(true_labels_encoded, 1 - predicted_clusters)
)

print(f'Accuracy: {accuracy * 100:.2f}%')

# رسم بياني باستخدام PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=predicted_clusters, cmap='viridis', s=50)
plt.title('product Clustered by Nutritional value Similarity')
plt.xlabel('Nutritional Component 1')
plt.ylabel('Nutritional Component 2')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
import pandas as pd

df=pd.read_csv('/content/BakedFoodNutritions_With_Target.csv')

df

In [ ]:
#frist checking for duplicated data

df.duplicated().value_counts()

In [ ]:
# second print the duplicated data

df [df.duplicated()]

In [ ]:
#checking for null val

df.isnull().sum()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# تحميل البيانات
df = pd.read_csv("/content/BakedFoodNutritions_With_Target.csv")

# اختيار الخصائص (features)
features = [
    'Calories-kcl', 'Protein-g', 'Carb-g', 'Fiber-g', 'Sugar-g',
    'Sodium-g', 'SaturatedFat-g', 'MonounsaturatedFat-g',
    'PolyunsaturatedFat-g', 'TransaFat-g'
]
X = df[features]

# تحويل القيم النصية في الهدف إلى أرقام
le = LabelEncoder()
y = le.fit_transform(df['Target'])  # Healthy = 0, Not Healthy = 1 (أو العكس)

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# تدريب نموذج Logistic Regression
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# التنبؤ بالقيم
y_pred = model.predict(X_test)

# حساب الدقة
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# تحميل البيانات
df = pd.read_csv("/content/BakedFoodNutritions_With_Target.csv")

# اختيار الخصائص (features)
features = [
    'Calories-kcl', 'Protein-g', 'Carb-g', 'Fiber-g', 'Sugar-g',
    'Sodium-g', 'SaturatedFat-g', 'MonounsaturatedFat-g',
    'PolyunsaturatedFat-g', 'TransaFat-g'
]
X = df[features]

# تحويل القيم النصية في الهدف إلى أرقام
le = LabelEncoder()
y = le.fit_transform(df['Target'])  # Healthy = 0, Not Healthy = 1 (أو العكس)

# تقسيم البيانات إلى تدريب واختبار
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# تدريب نموذج Decision Tree
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)

# التنبؤ بالقيم
y_pred = tree_model.predict(X_test)

# حساب الدقة
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
# Install required libraries
!pip install flask flask-cors ngrok pyngrok # add pyngrok
!pip install gensim
# Import necessary modules
from flask import Flask, request, jsonify
from flask_cors import CORS
import os
import glob
from pathlib import Path
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.utils import tokenize
import string
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import threading
from pyngrok import ngrok # this will work now

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable Cross-Origin Resource Sharing (CORS)

# Mount Google Drive for file access
from google.colab import drive
drive.mount("/content/drive")

# Directory path for text files
directory_path = '/content/drive/MyDrive/Colab Notebooks/Document'
TextFiles = glob.glob(f"{directory_path}/*.txt")
TextTitles = [Path(text).stem for text in TextFiles]

# Debug the file search
if not TextFiles:
    print("No text files found in the specified directory. Please verify the path and the files.")
else:
    print("Text files found:", TextFiles)

# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize preprocessing tools
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()
stopwords = set(nltk.corpus.stopwords.words('english'))


# Helper functions for Information Retriever
def normalize(text):
    t = list(tokenize(text.lower()))
    s = [word for word in t if word not in stopwords]
    p = [word for word in s if word not in string.punctuation]
    st = [ps.stem(word) for word in p]
    l = [lemmatizer.lemmatize(word) for word in st]
    return l

# Load and normalize documents
# Moved the normalize function definition before it's used in the loop
original_docs = []
normalized_docs = []

for filename in os.listdir(directory_path):
    txt_file = os.path.join(directory_path, filename)
    if os.path.isfile(txt_file) and filename.endswith(".txt"):
        with open(txt_file, encoding="utf-8") as f:
            txt_file_as_string = f.read()
            original_docs.append(txt_file_as_string)
            normalized_docs.append(normalize(txt_file_as_string))

def counting_docs(w, docs):
    count = 0
    for doc in docs:
        if w in doc:
            count += 1
    return count

def Calc_TF_IDF(document, wordDict):
    TF_IDF = []
    for r in range(len(document)):
        row = []
        for w in wordDict:
            TF = document[r].count(w) / len(document[r]) if len(document[r]) > 0 else 0
            IDF = math.log10(len(document) / counting_docs(w, document)) if counting_docs(w, document) > 0 else 0
            row.append(round(TF * IDF, 4))
        TF_IDF.append(row)
    return TF_IDF

def Calc_Q_IDF(query, docs, wordDict):
    Q_IDF = []
    for w in wordDict:
        if query.count(w) == 0:
            Q_IDF.append(float(0))
        else:
            IDF = math.log10(len(docs) / counting_docs(w, docs)) if counting_docs(w, docs) > 0 else 0
            Q_IDF.append(round(query.count(w) * IDF, 3))
    return Q_IDF

def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b) + 1e-10)

# Build word dictionary and calculate TF-IDF
flat_list = [item for sublist in normalized_docs for item in sublist]
wordDict = list(dict.fromkeys(flat_list))
TF_IDF = Calc_TF_IDF(normalized_docs, wordDict)

# Load data for Classification
file_path = '/content/drive/MyDrive/Colab Notebooks/Document/BakedFoodNutritions_With_Target.csv'
if os.path.exists(file_path):
    print("File found!")
else:
    raise FileNotFoundError("File not found. Please verify the path.")

# Load data for Classification
df = pd.read_csv(file_path)
features = [
    'Calories-kcl', 'Protein-g', 'Carb-g', 'Fiber-g', 'Sugar-g',
    'Sodium-g', 'SaturatedFat-g', 'MonounsaturatedFat-g',
    'PolyunsaturatedFat-g', 'TransaFat-g'
]
X = df[features]
y = df['Target'].apply(lambda x: 1 if x == "Healthy" else 0)  # Convert labels to binary
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Logistic Regression model
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

# Define endpoint for Information Retriever
@app.route('/search', methods=['POST'])
def search():
    try:
        query = request.json.get('query')
        if not query:
            return jsonify({"error": "Query is required"}), 400

        normalized_query = normalize(query)
        QTF = Calc_Q_IDF(normalized_query, normalized_docs, wordDict)
        doc_sim = []

        for r in range(len(normalized_docs)):
            cosine = cos_sim(QTF, TF_IDF[r])
            if cosine != 0:
                doc_sim.append([TextTitles[r], cosine])

        sorted_list = sorted(doc_sim, key=lambda x: x[1], reverse=True)
        return jsonify({"results": sorted_list})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# Define endpoint for Classification
@app.route('/classify', methods=['POST'])
def classify():
    try:
        data = request.json
        calories = float(data.get('calories'))
        sugar = float(data.get('sugar'))
        saturated_fat = float(data.get('saturated_fat'))

        if not all([calories, sugar, saturated_fat]):
            return jsonify({"error": "All features are required"}), 400

        input_features = [[calories, 0, 0, 0, sugar, 0, saturated_fat, 0, 0, 0]]
        prediction = model.predict(input_features)[0]
        result = "Healthy" if prediction == 1 else "Not Healthy"

        return jsonify({"result": result})
    except Exception as e:
        return jsonify({"error": str(e)}), 500
ngrok.set_auth_token("2uufMlSL0ow7ob72BCxneFd6TQy_3zjkyU6EUFp8hhFbPQLEY")  # Replace with your actual token

public_url = ngrok.connect(5000)  # Reconnect ngrok
print(f"Public URL: {public_url}")

app.run(port=5000)

In [ ]:
!curl -X POST https://e332-35-245-7-137.ngrok-free.app/search \
-H "Content-Type: application/json" \
-d '{"query": "sugar"}'